In [1]:
import sys
sys.path.append('/home/ubuntu/fast-autoaugment')
import torch
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel
from FastAutoAugment.networks import get_model
from theconf import Config as C
import random
import copy
from torchvision.transforms import transforms
import multiprocessing as mp
from tqdm import tqdm
import matplotlib.pyplot as plt

from FastAutoAugment.datasets import CIFAR10_mod 
from FastAutoAugment.metrics import accuracy
from FastAutoAugment.hardness_measures import AVH
from FastAutoAugment.augmentations import *

In [2]:
_ = C('/home/ubuntu/fast-autoaugment/confs/test.yaml')

In [3]:
model = get_model(C.get()['model'], 10, local_rank=-1)

In [4]:
## If trained model is needed. Specify the save_path of the required run.
save_path = '/efs-cotton/outputs/fast-autoaugment/confs/hardnessaware/wresnet28x10_rcifar-AA-rerun3/test.pth'
data = torch.load(save_path)
key = 'model' if 'model' in data else 'state_dict'

if 'epoch' not in data:
    model.load_state_dict(data)
else:
    if not isinstance(model, (DataParallel, DistributedDataParallel)):
        model.load_state_dict({k.replace('module.', ''): v for k, v in data[key].items()})
    else:
        model.load_state_dict({k if 'module.' in k else 'module.'+k: v for k, v in data[key].items()})

In [5]:
data['epoch']

195

In [6]:
data['log']

{'train': {'loss': 0.43232580375671387,
  'top1': 0.96325,
  'top5': 0.99775,
  'lr': 0.0006155829702431171},
 'valid': {'loss': 0.0, 'top1': 0.0, 'top5': 0.0},
 'test': {'loss': 0.5973012451648713, 'top1': 0.8451, 'top5': 0.9913}}

In [7]:
policy = {"ShearX0.15": [("ShearX", 1, 0.15)],
          "ShearX0.30": [("ShearX", 1, 0.30)],
          "ShearX0.60": [("ShearX", 1, 0.60)],
          "ShearX1": [("ShearX", 1, 1.0)],
          "ShearX1.5": [("ShearX", 1, 1.5)],
          "ShearY0.15": [("ShearY", 1, 0.15)],
          "ShearY0.30": [("ShearY", 1, 0.30)],
          "ShearY0.60": [("ShearY", 1, 0.60)],
          "ShearY1": [("ShearY", 1, 1.0)],
          "ShearY1.5": [("ShearY", 1, 1.5)],
          "TranslateX5": [("TranslateXAbs", 1, 5)],
          "TranslateX10": [("TranslateXAbs", 1, 10)],
          "TranslateX15": [("TranslateXAbs", 1, 15)],
          "TranslateX20": [("TranslateXAbs", 1, 20)],
          "TranslateY5": [("TranslateYAbs", 1, 5)],
          "TranslateY10": [("TranslateYAbs", 1, 10)],
          "TranslateY15": [("TranslateYAbs", 1, 15)],
          "TranslateY20": [("TranslateYAbs", 1, 20)],
          "AutoContrast": [("AutoContrast", 1, 1)],
          "Invert": [("Invert", 1, 1)],
          "Equalize": [("Equalize", 1, 1)],
          "Flip": [("Flip", 1, 1)],
          "Posterize0": [("Posterize", 1, 0)],
          "Posterize2": [("Posterize", 1, 2)],
          "Posterize4": [("Posterize", 1, 4)],
          "Posterize6": [("Posterize", 1, 6)],
          "Brightness0": [("Brightness", 1, 0)],
          "Brightness0.05": [("Brightness", 1, 0.05)],
          "Brightness0.1": [("Brightness", 1, 0.1)],
          "Brightness0.5": [("Brightness", 1, 0.5)],
          "Brightness1.5": [("Brightness", 1, 1.5)],
          "Brightness2": [("Brightness", 1, 2)],
          "Brightness3": [("Brightness", 1, 3)],
          "Color0": [("Color", 1, 0)],
          "Color0.05": [("Color", 1, 0.05)],
          "Color0.1": [("Color", 1, 0.1)],
          "Color0.5": [("Color", 1, 0.5)],
          "Color1.5": [("Color", 1, 1.5)],
          "Color2": [("Color", 1, 2)],
          "Color3": [("Color", 1, 3)],
          "Sharpness0": [("Sharpness", 1, 0)],
          "Sharpness0.05": [("Sharpness", 1, 0.05)],
          "Sharpness0.1": [("Sharpness", 1, 0.1)],
          "Sharpness0.5": [("Sharpness", 1, 0.5)],
          "Sharpness1.5": [("Sharpness", 1, 1.5)],
          "Sharpness2": [("Sharpness", 1, 2)],
          "Sharpness3": [("Sharpness", 1, 3)],
          }

In [8]:
class Augmentation2(object):
    def __init__(self, policies):
        self.policies = policies

    def __call__(self, img, hardness_score=None):
        for name, pr, level in self.policies:
            if random.random() > pr:
                continue
#             if level == 'random':
#                 level = random.choice(range(10))*0.1
            img = apply_augment_direct(img, name, level)
        return img

In [9]:
_CIFAR_MEAN, _CIFAR_STD = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
dataroot = "data"

In [10]:
no_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(_CIFAR_MEAN, _CIFAR_STD),
        ])
basic_dataset = CIFAR10_mod(root=dataroot, train=True, download=True, transform=no_transform)

Files already downloaded and verified


In [11]:
baseline_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(_CIFAR_MEAN, _CIFAR_STD),
        ])

In [12]:
policy_transforms = {}
for key, value in policy.items():
    transform = copy.deepcopy(baseline_transform)
    transform.transforms.insert(0, Augmentation2(policy[key]))
    policy_transforms[key] = transform

In [13]:
# policy_transforms['Rotatem30'].transforms[0].policies

In [14]:
policy_datasets = {}
for key, value in policy_transforms.items():
    policy_datasets[key] = CIFAR10_mod(root=dataroot, train=True, download=True, transform=policy_transforms[key])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files alread

In [15]:
basic_dataloader = torch.utils.data.DataLoader(basic_dataset, batch_size=16, shuffle=False, num_workers=0, drop_last=False)
policy_dataloaders = {}
for key, value in policy_datasets.items():
    policy_dataloaders[key] = torch.utils.data.DataLoader(
        policy_datasets[key], batch_size=16, shuffle=False, 
        num_workers=0, drop_last=False)

In [16]:
all_preds = []
all_embeddings = []
all_labels = []
all_indices = []
with torch.no_grad():
    model.eval()
    loader = tqdm(basic_dataloader, disable=False)
    for i, (data, label, index) in enumerate(loader):
        data, label = data.cuda(), label.cuda()
        preds, embeddings = model(data)

        all_preds.append(preds)
        all_embeddings.append(embeddings)
        all_labels.append(label)
        all_indices.append(index)
        top1, top5 = accuracy(preds, label, (1, 5))
        if i%1000==0:
            print("top1", top1, "top5", top5)
    del data, label, index, preds, embeddings

  0%|          | 4/3125 [00:01<48:56,  1.06it/s]  

top1 tensor(1., device='cuda:0') top5 tensor(1., device='cuda:0')


 32%|███▏      | 1004/3125 [00:52<01:49, 19.43it/s]

top1 tensor(0.8125, device='cuda:0') top5 tensor(1., device='cuda:0')


 64%|██████▍   | 2004/3125 [01:43<00:57, 19.42it/s]

top1 tensor(0.9375, device='cuda:0') top5 tensor(1., device='cuda:0')


 96%|█████████▌| 3004/3125 [02:34<00:06, 19.41it/s]

top1 tensor(0.8750, device='cuda:0') top5 tensor(1., device='cuda:0')


100%|██████████| 3125/3125 [02:40<00:00, 19.49it/s]


In [17]:
avh = AVH()

In [18]:
unaugmented_hardness = avh(model=model, embeddings=torch.cat(all_embeddings), 
                           targets=torch.cat(all_labels))

In [19]:
unaugmented_hardness = unaugmented_hardness.cpu().numpy()
all_indices = torch.cat(all_indices).cpu().numpy()

In [20]:
combined = list(zip(*sorted(zip(unaugmented_hardness, all_indices))))

In [21]:
sorted_indices = list(combined[1])

In [22]:
easy_indices = sorted_indices[0:200]
hard_indices = sorted_indices[-200:]

In [23]:
def process(model, dataloader):
    all_preds = []
    all_embeddings = []
    all_labels = []
    all_indices = []
    with torch.no_grad():
        model.eval()
        loader = tqdm(dataloader, disable=False)
        for i, (data, label, index) in enumerate(loader):
            data, label = data.cuda(), label.cuda()
            preds, embeddings = model(data)

            all_preds.append(preds)
            all_embeddings.append(embeddings)
            all_labels.append(label)
            all_indices.append(index)
            top1, top5 = accuracy(preds, label, (1, 5))
            if i%1000==0:
                print("top1", top1, "top5", top5)
        del data, label, index, preds, embeddings
    
    avh = AVH()
    hardness_scores = avh(model=model, embeddings=torch.cat(all_embeddings), 
                           targets=torch.cat(all_labels))
    return hardness_scores

In [24]:
policy_hardness_scores = {}
for key, value in policy_dataloaders.items():
    policy_hardness_scores[key] = process(model, policy_dataloaders[key])

  0%|          | 3/3125 [00:00<02:05, 24.80it/s]

top1 tensor(0.9375, device='cuda:0') top5 tensor(1., device='cuda:0')


 32%|███▏      | 1004/3125 [00:51<01:50, 19.24it/s]

top1 tensor(0.7500, device='cuda:0') top5 tensor(0.9375, device='cuda:0')


 64%|██████▍   | 2004/3125 [01:42<00:57, 19.35it/s]

...

top1 tensor(0.8125, device='cuda:0') top5 tensor(1., device='cuda:0')


 96%|█████████▌| 3004/3125 [02:33<00:06, 19.19it/s]

top1 tensor(0.8125, device='cuda:0') top5 tensor(1., device='cuda:0')


  0%|          | 0/3125 [00:00<?, ?it/s]


KeyError: 'Flip'

In [ ]:
len(policy_hardness_scores)

In [ ]:
easy_hardness_scores = {}
hard_hardness_scores = {}
easy_hardness_scores["unaugmented"] = unaugmented_hardness[easy_indices]
hard_hardness_scores["unaugmented"] = unaugmented_hardness[hard_indices]
for key, value in policy_hardness_scores.items():
    easy_hardness_scores[key] = value[easy_indices].cpu().numpy()
    hard_hardness_scores[key] = value[hard_indices].cpu().numpy()

In [ ]:
hardness_save_dir = save_path.replace('test.pth', 'hardness_scores')

In [ ]:
import os
if not os.path.exists(hardness_save_dir):
    os.makedirs(hardness_save_dir)

In [ ]:
hardness_save_path = os.path.join(hardness_save_dir, "exp4.pt")

In [ ]:
torch.save({'policy': policy, 'easy_hardness_scores': easy_hardness_scores, "hard_hardness_scores": hard_hardness_scores}, hardness_save_path)

In [ ]:
hardness_save_path